# Лабораторная работа 2 (МДТТ) #


In [1]:
import numpy as np
import math

In [2]:
MU = 0.2 #коэф пуассона
K = 9 #параметр кривизны
END = 8 #верхняя граница

PI = math.pi
sin = math.sin

In [26]:
W = []
Z = []

W_s, Z_s = [],[]

def init_reset():
    global W, Z, W_s, Z_s
    W = np.zeros((END,END),dtype='float128')
    Z = np.zeros((END,END),dtype='float128')
    W_s = np.zeros((END,END, 4),dtype='float128')
    Z_s = np.zeros((END,END, 4),dtype='float128')
    
def alpha():
    l, r = 0, 0
    for i in range(1, END):
        for j in range(1, END):
            l+= (i**2) * (W[i][j]**2)
            
    for i in range(1, END, 2):
        for j in range(1, END, 2):
            r+=W[i][j]/(i*j)
            
    return 3*(K**2)*(1+MU) * ((PI**2)/2*l - 16/(PI**2) *r)

def delta(i, j): 
    return int(i % 2 == 1 and j % 2 == 1)

def wij_tau(zij):
    return zij

def zij_tau(q, wij, i, j):
    return (q+ 2*alpha()) * 16 /(PI**2 * i**2 * j**2) * delta(i, j) - PI**2 *(i**2 + j**2)*(alpha() + PI**2 * (i**2 + j**2))* wij

def euler(q, h):
    for k in range(1000):
        for i in range(1,END, 2):
            for j in range(1,END,2):
                Z[i][j] += h * zij_tau(q, W[i][j], i, j)
                W[i][j] += h * wij_tau(Z[i][j])
    
    
def rk4(q, h, r = 1000):
    global W,Z
    for k in range(r):
        for i in range(1,END):
            for j in range(1,END):
                k1 = h * wij_tau(Z[i][j])
                m1 = h * zij_tau(q, W[i][j], i, j)
                k2 = h * wij_tau((Z[i][j] + m1/2))
                m2 = h * zij_tau(q, (W[i][j] + k1/2), i, j)
                k3 = h * wij_tau((Z[i][j] + m2/2))
                m3 = h * zij_tau(q, (W[i][j] + k2/2), i, j)
                k4 = h * wij_tau((Z[i][j]+m3))
                m4 = h * zij_tau(q, (W[i][j] + k3), i, j)
                W[i][j] += (k1+k2+k3+k4)/6
                Z[i][j] += (m1+m2+m3+m4)/6
                if r == 4:
                    W_s[i][j][k] = W[i][j]
                    Z_s[i][j][k] = Z[i][j]
    
                    
def ab4(q, h):
    k = 4
    rk4(q,h,4)
    f = zij_tau
    g = wij_tau
    
    for hh in range(k-1, 1000):
        for i in range(1,END, 2):
            for j in range(1,END, 2):
                Z[i][j] += (0.1*h)*(55/24*f(q, W_s[i][j][3], i, j) - 59/24*f(q, W_s[i][j][2], i, j) + 37/24*f(q, W_s[i][j][1], i, j) - 9/24*f(q, W_s[i][j][0], i, j))
                Z_s[i][j][0], Z_s[i][j][1], Z_s[i][j][2], Z_s[i][j][3] = Z_s[i][j][1], Z_s[i][j][2], Z_s[i][j][3], Z[i][j]
                
                
                W[i][j] += (0.1*h)*(55/24*g(Z_s[i][j][3]) - 59/24*g(Z_s[i][j][2]) + 37/24*g(Z_s[i][j][1]) - 9/24*g(Z_s[i][j][0]))
                W_s[i][j][0], W_s[i][j][1], W_s[i][j][2], W_s[i][j][3] = W_s[i][j][1], W_s[i][j][2], W_s[i][j][3], W[i][j]
    
    
def compute(method, range_q, h):
    
    for q in range_q:
        init_reset()
        method(q, h)
        w0 = 0
        for i in range(1,END, 2):
            for j in range(1,END, 2):
                w0+=W[i][j]*sin(i*PI/2)*sin(j*PI/2)
        print(q, w0)
                
h = 0.001

print("Эйлер")
compute(euler, np.arange(38.56, 38.65, 0.01), h)
print("Рунге-Кутта")
compute(rk4, np.arange(38.42, 38.50, 0.01), h)
# print("Адамс — Башфорт")
# compute(ab4, np.arange(39, 41, 0.01), h)

Эйлер
38.56 0.0044070844630208338863
38.57 0.0021420704470275234387
38.58 0.00022376981376833838916
38.589999999999996 -0.0013011158170372400215
38.599999999999994 -0.0023866045281031828608
38.60999999999999 -0.002988823333704302958
38.61999999999999 -0.00306756635637094143
38.62999999999999 -0.0025879609568194172906
38.639999999999986 -0.0015221569700921594244
Рунге-Кутта
38.42 0.001946309658069922957
38.43 0.00076929608970554018803
38.44 -0.00023346806651158092011
38.449999999999996 -0.0010444304696042446796
38.459999999999994 -0.0016457115734668376023
38.46999999999999 -0.0020193025276632725528
38.47999999999999 -0.0021472977245748759487
38.48999999999999 -0.0020121615243392979264
